In [1]:
import os
import sys
import subprocess
import shutil
import io

# --- 1. Xoá thư mục chromadb cũ nếu tồn tại ---
if os.path.isdir("chromadb"):
    print("Đang xóa thư mục chromadb cũ...")
    shutil.rmtree("chromadb")

# --- 3. Kiểm tra và cài đặt các thư viện cần thiết ---
def ensure_package(pkg_name, import_name=None):
    try:
        __import__(import_name or pkg_name)
    except ImportError:
        print(f"Thiếu thư viện '{pkg_name}', đang cài đặt...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg_name])
        __import__(import_name or pkg_name)

pkgs = [
    ("chromadb", None),
    ("langchain", None),
    ("ollama", None),
    ("tiktoken", None),
    ("PyPDF2", None)
]

for pkg, imp in pkgs:
    ensure_package(pkg, imp)

Đang xóa thư mục chromadb cũ...


In [2]:
# --- Đọc PDF với decoder UTF-8 để tránh lỗi tuple ---
import os
from PyPDF2 import PdfReader
from langchain.schema import Document

pdf_path = r"D:\Project_self\pdf_place\CleanCode.pdf"  # Thay đường dẫn tới file PDF của bạn
reader = PdfReader(pdf_path)
docs = []

for i, page in enumerate(reader.pages):
    raw_text = page.extract_text() or ""
    # Nếu raw_text là bytes, decode bằng utf-8 và bỏ ký tự không hợp lệ
    if isinstance(raw_text, (bytes, bytearray)):
        text = raw_text.decode("utf-8", errors="ignore")
    else:
        text = raw_text
    docs.append(Document(
        page_content=text,
        metadata={"source": f"{os.path.basename(pdf_path)}_page_{i+1}"}
    ))

In [3]:
from PyPDF2 import PdfReader
from langchain.schema import Document
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

In [4]:
# Sử dụng embedding model local
embed_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\klein\AppData\Local\Temp\ipykernel_43292\1977429318.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


In [5]:
# --- 7. Tạo hoặc tải lại vector database local với fallback khi embed thất bại ---
try:
    vectordb = Chroma.from_documents(
        documents=docs,
        embedding=embed_model,
        persist_directory="chromadb"
    )
    vectordb.persist()
    print("🎉 VectorDB khởi tạo thành công với model embedding hiện tại.")
    
except ValueError as e:
    print(f"[Warning] Embed thất bại: {e}")
    print("Chuyển sang model embedding thay thế: sentence-transformers/all-MiniLM-L6-v2")
    # Cài và dùng MiniLM embedding local thay thế
    from langchain.embeddings import SentenceTransformerEmbeddings
    fallback_embed = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    vectordb = Chroma.from_documents(
        documents=docs,
        embedding=fallback_embed,
        persist_directory="chromadb"
    )
    vectordb.persist()
    embed_model = fallback_embed  # cập nhật cho phần truy vấn sau
    print("🎉 VectorDB đã khởi tạo lại với MiniLM embedding.")

🎉 VectorDB khởi tạo thành công với model embedding hiện tại.


C:\Users\klein\AppData\Local\Temp\ipykernel_43292\4150500916.py:8: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [6]:
from dotenv import load_dotenv
import os, requests

load_dotenv()  # đọc các biến GEMINI_API_KEY, GEMINI_MODEL,…

API_KEY    = os.getenv("GEMINI_API_KEY")
MODEL_ID   = os.getenv("GEMINI_MODEL", "gemini-2.0-flash")
PROMPT     = os.getenv(
    "GEMINI_PROMPT",
    "Bạn là một trợ lý hữu ích. Hãy trả lời đầy đủ và chi tiết. Đừng lặp lại câu trả lời nếu không cần thiết."
)
MAX_TOKENS = int(os.getenv("GEMINI_MAX_TOKENS", "256"))
TEMP       = float(os.getenv("GEMINI_TEMPERATURE", "0.7"))

# Kiểm tra
print("API Key   :", "✓" if API_KEY else "✗")
print("Model     :", MODEL_ID)
print("MaxTokens :", MAX_TOKENS)
print("Temperature:", TEMP)



API Key   : ✓
Model     : gemini-2.0-flash
MaxTokens : 256
Temperature: 0.7


In [11]:
# Cell 3: Định nghĩa hàm call_gemini(prompt) – đảm bảo luôn return str
def call_gemini(prompt: str) -> str:
    if not API_KEY:
        raise ValueError("Please set GEMINI_API_KEY in .env")

    is_chat = MODEL_ID.lower().startswith("chat-")
    # Build endpoint + payload
    if is_chat:
        version = "v1beta"
        endpoint = ":generateMessage"
        body_key = "messages"
        body_val = [{"author": "user", "content": prompt}]
        payload = {
            body_key: body_val,
            "generationConfig": {"maxOutputTokens": MAX_TOKENS, "temperature": TEMP}
        }
    else:
        if MODEL_ID.endswith("-001"):
            version = "v1"
            endpoint = ":generateText"
            body_key = "prompt"
            body_val = {"text": prompt}
            payload = {
                body_key: body_val,
                "maxOutputTokens": MAX_TOKENS,
                "temperature": TEMP
            }
        else:
            version = "v1beta"
            endpoint = ":generateContent"
            body_key = "contents"
            body_val = [{"parts": [{"text": prompt}]}]
            payload = {
                body_key: body_val,
                "generationConfig": {"maxOutputTokens": MAX_TOKENS, "temperature": TEMP}
            }

    url = f"https://generativelanguage.googleapis.com/{version}/models/{MODEL_ID}{endpoint}?key={API_KEY}"
    resp = requests.post(url, json=payload, headers={"Content-Type":"application/json"}, timeout=30)
    resp.raise_for_status()
    data = resp.json()

    # Lấy candidate đầu tiên
    cand = data.get("candidates", [{}])[0]

    # Chat case: unwrap message.content.parts -> str
    if is_chat:
        msg = cand.get("message", {}).get("content", {})
        # msg có thể là string hoặc dict với parts
        if isinstance(msg, str):
            return msg
        parts = msg.get("parts", [])
        return "".join([p.get("text", "") for p in parts])

    # Text-only case: output or content
    out = cand.get("output", "") or cand.get("content", "")
    if isinstance(out, str):
        return out
    # Nếu out là dict (e.g. {parts:[...]})
    parts = out.get("parts", [])
    if parts:
        return "".join([p.get("text", "") for p in parts])
    # fallback
    return str(out)


In [12]:
# Cell 4: Test lại call_gemini
test_prompt = "Tóm tắt chương 1 của tài liệu PDF cho tôi."
print("👉 Prompt:", test_prompt)
print("\n🎉 Gemini trả lời:\n", call_gemini(test_prompt))

👉 Prompt: Tóm tắt chương 1 của tài liệu PDF cho tôi.

🎉 Gemini trả lời:
 Để tôi tóm tắt chương 1 của tài liệu PDF cho bạn, vui lòng cung cấp tài liệu PDF đó. Bạn có thể làm điều này bằng cách:

*   **Tải lên tệp PDF trực tiếp cho tôi.** (Nếu nền tảng bạn đang sử dụng cho phép)
*   **Cung cấp liên kết đến tệp PDF.** (Nếu nó có sẵn trực tuyến)
*   **Sao chép và dán nội dung của chương 1 vào khung chat.** (Nếu chương đó không quá dài)

Sau khi bạn cung cấp thông tin này, tôi sẽ đọc và tóm tắt chương 1 cho bạn.


In [31]:
# Cell 5: Định nghĩa wrapper GeminiLLM dùng call_gemini
from langchain.llms.base import LLM
from pydantic import BaseModel
from typing import Optional, List

class GeminiLLM(LLM, BaseModel):
    model_name: str
    api_key:    str
    max_output_tokens: int = 1000
    temperature:       float = 0.6

    @property
    def _llm_type(self) -> str:
        return "gemini"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        # Gọi hàm call_gemini đã định nghĩa ở Cell 3
        return call_gemini(prompt)

    async def _acall(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        return self._call(prompt, stop)

In [32]:
# Cell 6: Khởi tạo pipeline RAG với GeminiLLM và vectordb
from langchain.chains import RetrievalQA

# 1) Khởi tạo GeminiLLM
gemini_llm = GeminiLLM(
    model_name=MODEL_ID,            # từ Cell 2
    api_key=API_KEY,                # từ Cell 2
    max_output_tokens=MAX_TOKENS,   # từ Cell 2
    temperature=TEMP                # từ Cell 2
)

# 2) Tạo RetrievalQA chain
retriever = vectordb.as_retriever(search_kwargs={"k": 4})
qa_chain = RetrievalQA.from_chain_type(
    llm=gemini_llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# 3) Định nghĩa hàm ask để hỏi và in kết quả + nguồn
def ask(question: str):
    result = qa_chain.invoke({"query": question})
    print("=== ANSWER ===")
    print(result["result"])
    print("\n=== SOURCES ===")
    for doc in result["source_documents"]:
        print(f"- {doc.metadata.get('source', '')}")


In [30]:
ask("describe details as much as you can about Smell and Heuristic General.")

=== ANSWER ===
Based on the provided context, here's what I can tell you about "Smells and Heuristics" in general:

*   **Definition:** Smells and heuristics are indicators of potential problems in code. They represent observations and guidelines that experienced developers use to identify areas that might need refactoring or further attention.

*   **Origin:** The concept of "Code Smells" was popularized by Martin Fowler in his book "Refactoring." The text provided expands on Fowler's ideas and includes additional smells and heuristics.

*   **Purpose:** Smells and heuristics are intended to guide developers in writing, reading, and cleaning code. They help developers understand their reactions to code and provide a basis for making informed decisions about code quality and maintainability.

*   **Application:** The text emphasizes the importance of applying heuristics in the context of real-world code changes. The case studies in the book demonstrate how heuristics are used to make s